# **<font color="red"> This notebook is designed to run on kekcc. </font>**

In [3]:
using Condor
using NPZ
using Healpix
using Plots
using PyCall
using Falcons
using PyPlot
using DataFrames

hp = pyimport("healpy")
np = pyimport("numpy")
plt = pyimport("matplotlib.pyplot")

PyObject <module 'matplotlib.pyplot' from '/home/cmb/naganoy/.julia/conda/3/lib/python3.8/site-packages/matplotlib/pyplot.py'>

### Set the Healpix parameters to be used in the calculation

In [6]:
nside=128
npix = nside2npix(nside)
res = Resolution(nside)
lmax = 3*nside-1
mmax = 3*nside-1

### Here, as an example, the sky and beam used in PTEP are used for the calculations

In [2]:
alm_path="/gpfs/group/cmb/litebird/usr/wangw/d0s0/alm_LB_HFT_195_PTEP_20200915_compsep.fits"
blm_path="/gpfs/group/cmb/litebird/usr/wangw/output_wang/beam/blm/fc/H1-195/blm_H00_120_Q_195B_I000.fits"
alm_LB_HFT_195_PTEP_20200915_compsep=ComplexF64.(hp.read_alm(alm_path, hdu = [1,2,3]))
blm_H00_120_Q_195B_I000=ComplexF64.(hp.read_alm(blm_path, hdu = [1,2,3]))

3×134655 Matrix{ComplexF64}:
 0.101997+0.0im  0.176581+0.0im  0.227776+0.0im  …  1.34001e-9+8.83946e-10im
      0.0+0.0im       0.0+0.0im       0.0+0.0im            0.0+0.0im
      0.0+0.0im       0.0+0.0im       0.0+0.0im            0.0+0.0im

### In PTEP, nside=512 for both sky and beam, plus lmax=1024 and mmax=140 for beam. 
### The operation is performed to change this to correspond to nside=128, which is used in this case.

In [4]:
function truncate_alm(alm, lmax, mmax)
    alm_new = Alm(lmax,mmax)
    size = numberOfAlms(lmax, min(mmax,alm.mmax))
    for idx in 1:size
        m = convert(Int64, ceil(((2 * lmax + 1) - sqrt((2 * lmax + 1) ^ 2 - 8 * (idx - 1 - lmax))) / 2))
        l = idx - 1 - m * div(2 * lmax + 1 - m,2)
        alm_new.alm[idx] = alm.alm[almIndex(alm, l, m)]
    end
    return alm_new
end

py"""def truncate_alm(alms, lmax = -1, mmax = -1, mmax_in=-1):
    import healpy as hp
    import numpy as np
    l2max = hp.Alm.getlmax(alms.shape[-1], mmax=mmax_in)
    if lmax != -1 and lmax > l2max:
        raise ValueError("Too big lmax in parameter")
    elif lmax == -1:
        lmax = l2max

    if mmax_in == -1:
        mmax_in = l2max

    if mmax == -1:
        mmax = lmax
    if mmax > mmax_in:
        mmax = mmax_in

    # if out_dtype is None:
    #     out_dtype = alms[0].real.dtype

    l, m = hp.Alm.getlm(lmax)
    idx = np.where((l <= lmax) * (m <= mmax))
    l = l[idx]
    m = m[idx]

    idx_in_original = hp.Alm.getidx(l2max, l=l, m=m)
    
    return alms[..., idx_in_original]
"""

py"""def pad_alm(alms, mmax_in = -1):
    import healpy as hp
    import numpy as np
    lmax = hp.Alm.getlmax(alms.shape[-1], mmax=mmax_in)
    lm_size = hp.Alm.getsize(lmax)
    alms_new = np.zeros((alms.shape[:-1])+(lm_size,), dtype=alms.dtype)
    alms_new[..., :alms.shape[-1]] = alms
    return alms_new 
"""

In [7]:
pad_alm=py"pad_alm"(alm_LB_HFT_195_PTEP_20200915_compsep, mmax_in=3*512)
alm=py"truncate_alm"(pad_alm, lmax = lmax, mmax = lmax)
pad_blm=py"pad_alm"(blm_H00_120_Q_195B_I000, mmax_in=140)
blm=py"truncate_alm"(pad_blm, lmax = lmax, mmax = mmax)
unique_θ = unique_theta(npix, res);

np.save("alm_test", alm)
np.save("blm_test",blm)

### Specify the PATH of parameters such as alm

In [13]:
alm_path = "./test_alm.npy"
blm_path = "./test_blm.npy"
dir_path = "./hoge"

"./hoge"

In [16]:
open( "convolve_T.jl", "w" ) do fp
    
write( fp, """using Condor
using NPZ
using Healpix
using DataFrames
using Falcons
using PyCall
np=pyimport("numpy")

# standard input
idx=parse(Int64, ARGS[1]) # Specifies a piece of sky with unique θ
nside = parse(Int64, ARGS[2]) # Specify nside
dir_save = ARGS[3] # Specify preconvolved-map path
alm_path=ARGS[4]
blm_path=ARGS[5]
        
# path of save file
dir=dir_save*"test_\$idx.hdf5"

# reading of input parameters
alm = npzread(alm_path)
blm = npzread(blm_path)
        
# Healpix-related parameters
npix = nside2npix(nside)
res = Resolution(nside)
lmax = 3nside-1

# Get unique θ value determined by nside
unique_θ = unique_theta(npix, res);

# Perform preconvolution
FFTConv_demo_onlyT(alm, blm, unique_θ, lmax, nside, idx, dir)
""" )
end

open( "job_convolve.sh", "w" ) do fp
    
write( fp, """
script=convolve_T.jl
count=1
nside=$nside
dir=$dir_path
alm_path=$alm_path 
blm_path=$blm_path 
countstop=\$(($nside*4-1))
for i in `seq  1 \$countstop`
do
    if [ \$(( \$count % 10 )) -eq 0 ] ; then
    sleep 1
    echo "sleep!"
    fi
    bsub -q cmb_px julia \$script \$i \$nside \$dir \$alm_path \$blm_path 
    count=\$((++count)) 
done
""" )
    
end


333

In [ ]:
run(`bash job_convolve.sh`)

### Specify sampling rate

In [22]:
Hz=1

1

In [23]:
open( "scan_T.jl", "w" ) do fp
    
write( fp, """using Condor
using NPZ
using Healpix
using DataFrames
using Falcons
using PyCall
np=pyimport("numpy")

# standard input
idx=parse(Int64, ARGS[1]) # Specifies a piece of sky with unique θ
nside = parse(Int64, ARGS[2]) # Specify nside
dir_save = ARGS[3] # Specify preconvolved-map path
Hz=parse(Int64, ARGS[4]) # Specify Hz of scan
        
# path of save file
dir=dir_save*"test_\$idx.hdf5"
dir_map=dir_save*"test_map=\$idx=\$nside=\$Hz"

# Healpix-related parameters
npix = nside2npix(nside)
res = Resolution(nside)
lmax = 3nside-1

# Falcons-related parameters
ss = gen_ScanningStrategy()
day = 60 * 60 * 24
year = day * 365
ss.nside = nside
ss.duration = year #[sec]
ss.sampling_rate = Hz #[Hz]
ss.alpha = 45 #[degree]
ss.beta = 50 #[degree]
ss.prec_rpm = period2rpm(192.348)
ss.spin_rpm = 0.05 #[rpm]
ss.hwp_rpm = 0.0 #[rpm]
ss.start_point = "pole" #You can choose "pole" or "equator"
ss.coord="G"  # 
ss.FP_theta = [0.7378010233475734] #[target_det.theta[1]]
ss.FP_phi = [59.808181651844826] #[target_det.phi[1]] .+ 30

d_var , h= get_psi_make_TOD_T(ss, division = 1600, idx = idx, map_div=4, dir=dir)

np.save(dir_map, d_var[:,1])

#run(`rm \$dir`)
""" )
end

open( "job_scan.sh", "w" ) do fp
    
write( fp, """
script=scan_T.jl
count=1
nside=$nside
dir=$dir_path
Hz=$Hz
countstop=\$(($nside*4-1))
for i in `seq  1 countstop`
do
    if [ \$(( \$count % 10 )) -eq 0 ] ; then
    sleep 1
    echo "sleep!"
    fi
    bsub -q l julia \$script \$i \$nside \$dir \$Hz
    count=\$((++count)) 
done
""" )
    
end

261

In [299]:
run(`bash job_scan.sh`)

Job <60522458> is submitted to queue <cmb_px>.
Job <60522459> is submitted to queue <cmb_px>.
Job <60522462> is submitted to queue <cmb_px>.
Job <60522464> is submitted to queue <cmb_px>.
Job <60522466> is submitted to queue <cmb_px>.
Job <60522469> is submitted to queue <cmb_px>.
Job <60522471> is submitted to queue <cmb_px>.
Job <60522474> is submitted to queue <cmb_px>.
Job <60522475> is submitted to queue <cmb_px>.
sleep!
Job <60522484> is submitted to queue <cmb_px>.
Job <60522487> is submitted to queue <cmb_px>.
Job <60522489> is submitted to queue <cmb_px>.
Job <60522491> is submitted to queue <cmb_px>.
Job <60522494> is submitted to queue <cmb_px>.
Job <60522496> is submitted to queue <cmb_px>.
Job <60522499> is submitted to queue <cmb_px>.
Job <60522501> is submitted to queue <cmb_px>.
Job <60522503> is submitted to queue <cmb_px>.
Job <60522506> is submitted to queue <cmb_px>.
sleep!
Job <60522517> is submitted to queue <cmb_px>.
Job <60522519> is submitted to queue <cmb_px>.

Process(`bash job_sample.sh`, ProcessExited(0))

### Make a map

In [7]:
map_test = zeros(npix)
for i in 1:4nside-1
    try
        temp = npzread(dir_path*"/test_map=$i=$nside=$Hz"*".npy")
        pixmin,pixmax=unique_theta_detect(i, nside,npix)
        map_test[pixmin:pixmax] = real(temp[:,1])
    catch
        println("$i")
    end
end

In [8]:
hp.mollview(np.log10(map_test), cmap = "coolwarm", min=0,max=3, rot = (0,0))